In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My\ Drive/Colab\ Notebooks/Bert_Scratch

/content/drive/My Drive/Colab Notebooks/Bert_Scratch


In [0]:
!git clone https://github.com/google-research/bert.git

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
!unzip the file
!unzip cased_L-12_H-768_A-12.zip

In [0]:
cd cased_L-12_H-768_A-12

/content/drive/My Drive/Colab Notebooks/Bert_Scratch/cased_L-12_H-768_A-12


In [0]:
import os
os.mkdir("bert_output")
os.mkdir("data")

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [0]:
cd ..

/content/drive/My Drive/Colab Notebooks/Bert_Scratch


In [0]:
cd bert

/content/drive/My Drive/Colab Notebooks/Bert_Scratch/bert


In [0]:
df_train = pd.read_csv('data/sms.csv',names=['label','text'],encoding = "UTF-8", quotechar='"')
df_train["text"] = df_train['text'].str.replace('[^\w\s]','')
df_train = df_train.dropna()

In [0]:
df_train.reset_index(drop=True,inplace=True)

In [0]:
df_train['id'] = df_train.index
df_train['id'] = df_train['id'].map(str)+'_'+df_train.label
df_train['label'] = df_train['label'].map({'ham': 0, 'spam': 1})

df_bert = pd.DataFrame({'guid': df_train['id'],
    'label': df_train['label'],
    'alpha': ['a']*df_train.shape[0],
    'text': df_train['text']})
df_bert_train, df_bert_dev, df_test = np.split(df_bert.sample(frac=1), [int(.6*len(df_bert)), int(.8*len(df_bert))])

# =============================================================================
df_bert_test = pd.DataFrame({'guid': df_test['guid'],
    'text': df_test['text']}).reset_index(drop=True)
df_bert_train.to_csv('data/train.tsv', sep='\t', index=False, header=False, encoding="UTF-8")
df_bert_dev.to_csv('data/dev.tsv', sep='\t', index=False, header=False, encoding="UTF-8")
df_bert_test.to_csv('data/test.tsv', sep='\t', index=False, header=True, encoding="UTF-8")


In [0]:

!python run_classifier.py --task_name=cola --do_train=true --do_eval=true --data_dir=./data --vocab_file=./model/vocab.txt --bert_config_file=./model/bert_config.json --init_checkpoint=./model/bert_model.ckpt --max_seq_length=64 --train_batch_size=2 --learning_rate=2e-5 --num_train_epochs=3.0 --output_dir=./bert_output/ --do_lower_case=False --save_checkpoints_steps 10000


In [0]:
!python run_classifier.py --task_name=cola --do_predict=true --data_dir=./data --vocab_file=./model/vocab.txt --bert_config_file=./model/bert_config.json --init_checkpoint=./bert_output/model.ckpt-4983 --max_seq_length=64 --output_dir=./bert_output/

In [0]:
df_bert_test

In [0]:
df_test =df_bert_test
#read the results data for the probabilities
df_result = pd.read_csv('bert_output/test_results.tsv', sep='\t', header=None)#create a new dataframe
df_map_result = pd.DataFrame({'guid': df_test['guid'],
    'text': df_test['text'],
    'label': df_result.idxmax(axis=1)})#view sample rows of the newly created dataframe
df_map_result.sample(10)